# Weekly Report for Hospital Data

In [ ]:
# Imports
import sys
import pandas as pd
import psycopg

from credentials import DB_PASSWORD, DB_USER

In [5]:
#parameters
week = sys.argv[1]

In [4]:
# Print the week information of this report
print("This is the Hospital Data report for week:", week)

This is the Hospital Data report for week: 
